# Projet Knowledge Extraction - Partie A : Preprocessing et Représentation Text

**Université Paris Cité - Master 2 VMI**
**Cours :** IFLCE085 Recherche et extraction sémantique à partir de texte (Prof. Salima Benbernou)

**Équipe :**
- **Partie A (Preprocessing) : [Votre Nom]**
- Partie B : Boutayna EL MOUJAOUID
- Partie C : Franz Dervis
- Partie D : Aya Benkabour

---

## Étape 1 : Setup et Importations
Objectif : Configurer l'environnement et importer les librairies nécessaires.

In [ ]:
import sys
import os
import json
import pandas as pd
import numpy as np
import nltk
import spacy
import preprocessing as pp # Notre module local

# Configuration de l'affichage pandas
pd.set_option('display.max_colwidth', 100)

print("Environnement configuré avec succès.")

## Étape 2 : Chargement et Exploration des Données (SciREX)
Objectif : Charger le dataset SciREX (fichiers JSONL), analyser la structure des documents et vérifier la volumétrie.

In [ ]:
DATA_DIR = "release_data"
FILES = {
    "train": os.path.join(DATA_DIR, "train.jsonl"),
    "dev": os.path.join(DATA_DIR, "dev.jsonl"),
    "test": os.path.join(DATA_DIR, "test.jsonl")
}

def load_jsonl(file_path):
    """Charge un fichier JSONL dans une liste de dictionnaires."""
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    return data

# Chargement des données
print("Chargement des données...")
train_data = load_jsonl(FILES["train"])
dev_data = load_jsonl(FILES["dev"])
test_data = load_jsonl(FILES["test"])

print(f"Nombre de documents Train : {len(train_data)}")
print(f"Nombre de documents Dev   : {len(dev_data)}")
print(f"Nombre de documents Test  : {len(test_data)}")

In [ ]:
# Exploration d'un document type
doc_example = train_data[0]
print("\nClés disponibles dans un document :")
print(list(doc_example.keys()))

print("\nExemple de contenu (champs principaux) :")
print(f"ID: {doc_example.get('doc_id')}")
# SciREX contient souvent une liste de mots sous 'words' ou le texte brut
# Vérifions la structure du texte
if 'words' in doc_example:
    print(f"Début du texte (50 premiers mots) : {' '.join(doc_example['words'][:50])}...")
